## Density and Velocity (method D)


In [ ]:
d = parse(jpsreport_inifile)
method_D = d.getElementsByTagName("method_D")[0].attributes.items()[0][1]
geo_filename = os.path.join(
    os.path.dirname(jpsreport_inifile),
    d.getElementsByTagName("geometry")[0].attributes.items()[0][1],
)
cfiles = glob.glob(classical_fd_files)
ifiles = glob.glob(IFD_files)
ids_IFD = set([])
ids_FD = set([])
for f in ifiles:
    ids_IFD.add(int(f.split("id_")[-1].split(".")[0]))

sorted(ids_IFD)
for f in cfiles:
    ids_FD.add(int(f.split("id_")[-1].split(".")[0]))

sorted(ids_FD)
# fps
o = open(cfiles[0])
header = o.readline()
o.close()
fps = float(header.split(":")[-1])
# plot parameter
alpha = 0.3
color = "black"

In [ ]:
def plot_density_velocity(Id, frange):
    """
    method D
    file contain three columns:
    >> frame density velocity <<
    """
    fmin, fmax = frange
    files = glob.glob(
        os.path.join(
            jpsreport_ini_dir,
            output_dir,
            "Fundamental_Diagram",
            "Classical_Voronoi",
            "*id_{}.dat".format(Id),
        )
    )
    fig, axs = plt.subplots(2, 2)
    for f in files:
        data = np.loadtxt(f)
        mask = (data[:, 0] >= fmin) & (data[:, 0] <= fmax)
        frames = data[:, 0]
        # rho(t)
        axs[0][0].plot(
            [fmin], [data[fmin - int(min(frames)), 1]], "o", color=color
        )
        axs[0][0].plot(
            [fmax], [data[fmax - int(min(frames)), 1]], "o", color=color
        )
        axs[0][0].plot(data[:, 0], data[:, 1], "k")
        axs[0][0].plot(
            data[fmin - int(min(frames)) : fmax - int(min(frames)), 0],
            data[fmin - int(min(frames)) : fmax - int(min(frames)), 1],
            "-r",
            lw=2,
        )
        axs[0][0].set(xlabel=r"$frame$", ylabel="$\\rho\; [m{-2}$]")
        x0, x1 = axs[0][0].get_xlim()
        y0, y1 = axs[0][0].get_ylim()
        axs[0][0].set_aspect(abs(x1 - x0) / abs(y1 - y0))
        # v(t)
        axs[0][1].plot(data[:, 0], data[:, 2], "k")
        axs[0][1].plot(
            [fmin], [data[fmin - int(min(frames)), 2]], "o", color=color
        )
        axs[0][1].plot(
            [fmax], [data[fmax - int(min(frames)), 2]], "o", color=color
        )
        axs[0][1].plot(
            data[fmin - int(min(frames)) : fmax - int(min(frames)), 0],
            data[fmin - int(min(frames)) : fmax - int(min(frames)), 2],
            "-r",
            lw=2,
        )
        axs[0][1].set(xlabel=r"$frame$", ylabel="$v\; [m/s$]")
        x0, x1 = axs[0][1].get_xlim()
        y0, y1 = axs[0][1].get_ylim()
        axs[0][1].set_aspect(abs(x1 - x0) / abs(y1 - y0))
        # rho vs J
        data2 = data[mask]
        maxRho = np.max(data[:, 1])
        maxJ = np.max(data[:, 1] * data[:, 2])
        axs[1][1].scatter(
            data2[:, 1], data2[:, 1] * data2[:, 2], color=color, alpha=alpha
        )
        axs[1][1].set(
            xlabel="$\\rho\; [1/m{-2}]$",
            ylabel="$J\; [1/ms$]",
            xlim=[0, maxRho],
            ylim=[0, maxJ],
        )
        x0, x1 = axs[1][1].get_xlim()
        y0, y1 = axs[1][1].get_ylim()
        axs[1][1].set_aspect(abs(x1 - x0) / abs(y1 - y0))
        # axs[1][1].set_aspect('scaled')
        # rho vs v
        maxV = np.max(data[:, 2])
        axs[1][0].scatter(data2[:, 1], data2[:, 2], color=color, alpha=alpha)
        axs[1][0].set(
            xlabel="$\\rho\; [1/m{-2}]$",
            ylabel="$v\; [m/s$]",
            xlim=[0, maxRho],
            ylim=[0, maxV],
        )
        x0, x1 = axs[1][0].get_xlim()
        y0, y1 = axs[1][0].get_ylim()
        axs[1][0].set_aspect(abs(x1 - x0) / abs(y1 - y0))
    plt.tight_layout()

In [ ]:
def plot_IFD_density_velocity(Id, pid, frange):
    """
    method D
    file contain many columns:
    Frame PersID x, y, z, density, velocity, Voronoi Polygon, Intersection Polygon
    """
    fmin, fmax = frange
    fig, axs = plt.subplots(2, 2)
    files = glob.glob(
        os.path.join(
            jpsreport_ini_dir,
            output_dir,
            "Fundamental_Diagram",
            "IndividualFD",
            "*id_{}.dat".format(Id),
        )
    )

    for f in files:
        df = read_IFD(f)
        mask = (df["Frame"] >= fmin) & (df["Frame"] <= fmax)

        maxRho = np.max(df["rho"])
        maxVel = np.max(df["vel"])
        maxF = np.max(df["Frame"])
        minF = np.min(df["Frame"])
        if pid != "all":
            df = df[df["PersID"] == pid]
        else:
            df.mask(~mask, inplace=True)

        # rho(t)
        if pid != "all":
            axs[0, 0].plot(df["Frame"], df["rho"], color=color)
        else:
            axs[0, 0].scatter(
                df["Frame"], df["rho"], lw=0, color="black", alpha=alpha
            )

        axs[0, 0].set(
            xlabel=r"$frame$",
            ylabel="$\\rho\; [1/m{-2}$]",
            ylim=[0, maxRho],
            xlim=[minF, maxF],
        )
        x0, x1 = axs[0, 0].get_xlim()
        y0, y1 = axs[0, 0].get_ylim()
        axs[0, 0].set_aspect(abs(x1 - x0) / abs(y1 - y0))
        # v(t)
        if pid != "all":
            axs[0, 1].plot(df["Frame"], df["vel"], color=color)
        else:
            axs[0, 1].scatter(
                df["Frame"], df["vel"], lw=0, color="black", alpha=alpha
            )

        axs[0, 1].set(
            xlabel=r"$frame$",
            ylabel="$v\; [m/s$]",
            ylim=[0, maxVel],
            xlim=[minF, maxF],
        )
        x0, x1 = axs[0, 1].get_xlim()
        y0, y1 = axs[0, 1].get_ylim()
        axs[0, 1].set_aspect(abs(x1 - x0) / abs(y1 - y0))
        # rho(J)
        J = df["rho"] * df["vel"]
        maxJ = np.max(J)
        axs[1, 1].scatter(df["rho"], J, lw=0, color="black", alpha=alpha)
        axs[1, 1].set(
            xlabel="$\\rho\; [1/m{-2}]$",
            ylabel="$J\; [1/ms$]",
            ylim=[0, maxJ],
            xlim=[0, maxRho],
        )
        x0, x1 = axs[1, 1].get_xlim()
        y0, y1 = axs[1, 1].get_ylim()
        axs[1, 1].set_aspect(abs(x1 - x0) / abs(y1 - y0))

        axs[1, 0].scatter(
            df["rho"], df["vel"], lw=0, color="black", alpha=alpha
        )
        axs[1, 0].set(
            xlabel="$\\rho\; [1/m{-2}]$",
            ylabel="$v\; [m/s$]",
            ylim=[0, maxVel],
            xlim=[0, maxRho],
        )
        x0, x1 = axs[1, 0].get_xlim()
        y0, y1 = axs[1, 0].get_ylim()
        axs[1, 0].set_aspect(abs(x1 - x0) / abs(y1 - y0))

    plt.tight_layout()

In [ ]:
def plot_profiles(Id):
    density_files = os.path.join(
        field_dir, "density", "Prf_*id_{}_*".format(Id)
    )
    velocity_files = os.path.join(
        field_dir, "velocity", "Prf_*id_{}_*".format(Id)
    )
    v_Voronoi = glob.glob(velocity_files)
    f_Voronoi = glob.glob(density_files)
    # get the shape of the matrices
    shape = np.shape(np.loadtxt(f_Voronoi[0]))
    density = np.zeros(shape)
    velocity = np.zeros(shape)
    geo_filename = os.path.join(
        os.path.dirname(jpsreport_inifile),
        d.getElementsByTagName("geometry")[0].attributes.items()[0][1],
    )
    xml_datei = open(geo_filename, "r")
    geo_xml = parse(xml_datei)
    xml_datei.close()
    geominX, geomaxX, geominY, geomaxY = geo_limits(geo_xml)

    geometry_wall = read_subroom_walls(geo_xml)
    geometry_obst = read_obstacle(geo_xml)
    #  -------- density
    for density_file in f_Voronoi:
        if os.path.exists(density_file):
            density += np.loadtxt(density_file)

    density = density / len(f_Voronoi)
    #  --------- velocity
    for velocity_file in v_Voronoi:
        if os.path.exists(velocity_file):
            velocity += np.loadtxt(velocity_file)

    velocity = velocity / len(f_Voronoi)
    flow = density * velocity

    # plot
    figs, axs = plt.subplots(3, 1)
    axs[0].set_aspect("equal")

    for g in geometry_obst.keys():
        axs[0].add_patch(ppolygon(geometry_obst[g], color="white"))
        axs[1].add_patch(ppolygon(geometry_obst[g], color="white"))
        axs[2].add_patch(ppolygon(geometry_obst[g], color="white"))

    for gw in geometry_wall.keys():
        axs[0].plot(
            geometry_wall[gw][:, 0],
            geometry_wall[gw][:, 1],
            color="white",
            lw=1,
        )
        axs[1].plot(
            geometry_wall[gw][:, 0],
            geometry_wall[gw][:, 1],
            color="white",
            lw=1,
        )
        axs[2].plot(
            geometry_wall[gw][:, 0],
            geometry_wall[gw][:, 1],
            color="white",
            lw=1,
        )

    im1 = axs[0].imshow(
        density,
        cmap=cm.jet,
        interpolation="nearest",
        origin="lower",
        vmin=0,
        vmax=4,  # np.amax(density)
        extent=[geominX, geomaxX, geominY, geomaxY],
    )

    im2 = axs[1].imshow(
        velocity,
        cmap=cm.jet,
        interpolation="nearest",
        origin="lower",
        vmin=0,
        vmax=1.2,  # np.amax(velocity)
        extent=[geominX, geomaxX, geominY, geomaxY],
    )

    im3 = axs[2].imshow(
        flow,
        cmap=cm.jet,
        interpolation="nearest",
        origin="lower",
        vmin=0,
        vmax=2,  # np.amax(flow)
        extent=[geominX, geomaxX, geominY, geomaxY],
    )

    axs[0].set_xlabel("x [m]")
    axs[0].set_ylabel("y [m]")
    divider1 = make_axes_locatable(axs[0])
    cax1 = divider1.append_axes("right", size="3.5%", pad=0.3)
    cb1 = plt.colorbar(im1, cax=cax1)
    cb1.set_label("Density [$m^{-2}$]")

    axs[1].set_xlabel("x [m]")
    axs[1].set_ylabel("y [m]")
    divider2 = make_axes_locatable(axs[1])
    cax2 = divider2.append_axes("right", size="3.5%", pad=0.3)
    cb2 = plt.colorbar(im2, cax=cax2)
    cb2.set_label("Velocity [$m/s$]")

    axs[2].set_xlabel("x [m]")
    axs[2].set_ylabel("y [m]")
    divider3 = make_axes_locatable(axs[2])
    cax3 = divider3.append_axes("right", size="3.5%", pad=0.3)
    cb3 = plt.colorbar(im3, cax=cax3)
    cb3.set_label("Flow [$1/m \cdot s$]")
    plt.tight_layout()

In [ ]:
def get_ids():
    pids = {}
    for Id in ids_IFD:
        files = glob.glob(
            os.path.join(
                jpsreport_ini_dir,
                output_dir,
                "Fundamental_Diagram",
                "IndividualFD",
                "*id_{}.dat".format(Id),
            )
        )  # len(files) is actually 1

        f = files[0]
        df = read_IFD(f)
        p = ["all"] + sorted(pd.unique(df["PersID"]))
        pids[str(Id)] = p
    return pids

In [ ]:
def get_frame_limits(Id):
    files = glob.glob(
        os.path.join(
            jpsreport_ini_dir,
            output_dir,
            "Fundamental_Diagram",
            "Classical_Voronoi",
            "*id_{}.dat".format(Id),
        )
    )
    f = files[0]
    res = {}
    data = np.loadtxt(f)
    frames = data[:, 0]
    m = np.min(frames)
    M = np.max(frames)
    return m, M

In [ ]:
def IFD_plot_polygon_rho(Id, frame):
    m = d.getElementsByTagName("measurement_areas")
    verteces = []
    for o_num, o_elem in enumerate(m[0].getElementsByTagName("area_B")):
        if o_elem.getAttribute("id") == str(Id):
            n_vertex = len(o_elem.getElementsByTagName("vertex"))
            verteces = np.zeros((n_vertex, 2))

            for v_num, v_elem in enumerate(
                o_elem.getElementsByTagName("vertex")
            ):
                verteces[v_num, 0] = (
                    o_elem.getElementsByTagName("vertex")[v_num]
                    .attributes["x"]
                    .value
                )
                verteces[v_num, 1] = (
                    o_elem.getElementsByTagName("vertex")[v_num]
                    .attributes["y"]
                    .value
                )

    xml_datei = open(geo_filename, "r")
    geo_xml = parse(xml_datei)
    xml_datei.close()

    geometry_wall = read_subroom_walls(geo_xml)
    geometry_obst = read_obstacle(geo_xml)

    files = glob.glob(
        os.path.join(
            jpsreport_ini_dir,
            output_dir,
            "Fundamental_Diagram",
            "IndividualFD",
            "*id_{}.dat".format(Id),
        )
    )

    df_poly = pd.read_csv(
        files[0],
        comment="#",
        sep="\t",
        names=["Frame", "PersID", "x", "y", "z", "rho", "vel", "poly"],
        index_col=False,
    )

    # set the index to be this and don't drop
    df_poly.set_index(keys=["Frame"], drop=False, inplace=True)

    fig, ax = plt.subplots()
    ax.set_aspect("equal")

    for g in geometry_obst.keys():
        ax.add_patch(ppolygon(geometry_obst[g], color="gray"))

    for gw in geometry_wall.keys():
        ax.plot(
            geometry_wall[gw][:, 0],
            geometry_wall[gw][:, 1],
            color="black",
            lw=2,
        )

    sm = cm.ScalarMappable(cmap=cm.get_cmap("rainbow"))

    df_0 = df_poly[df_poly.Frame == frame]
    polys = df_0["poly"]
    rhos = df_0["rho"]
    X = df_0["x"]
    Y = df_0["y"]
    sm.set_clim(vmin=0, vmax=6)  # todo max rho
    for x, y, rho, poly in zip(X, Y, rhos, polys):
        p = str_to_array(poly) / 10000
        patch = ppolygon(p, fc=sm.to_rgba(rho), ec="white", lw=1)
        ax.add_patch(patch)

    # plot measurement area
    plt.plot(verteces[:, 0], verteces[:, 1], "-r")

    plt.axis("off")
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes

    axins = inset_axes(
        ax,
        width="5.5%",  # width = 5% of parent_bbox width
        height="100%",  # height : 50%
        loc="lower left",
        bbox_to_anchor=(1, 0.0, 1, 1),
        bbox_transform=ax.transAxes,
        borderpad=0,
    )
    fig.colorbar(sm, ax=ax, cax=axins)

In [ ]:
def plot_method_D(choice):
    style = {"description_width": "initial"}
    l = widgets.Layout(flex="0 1 auto", width="160px")

    if choice == "FD":
        fm, fM = get_frame_limits(list(ids_FD)[0])
        frameRange = widgets.IntRangeSlider(
            value=[fm, fM],
            min=fm,
            max=fM,
            step=1,
            description="Frame range",
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        MeasurementArea = widgets.Dropdown(
            options=ids_FD,
            layout=l,
            style=style,
            description="Measurement area:",
            disabled=False,
        )

        def change_f(*args):
            fm, fM = get_frame_limits(MeasurementArea.value)
            frameRange.min = fm
            frameRange.max = fM
            frameRange.value = [fm, fM]

        MeasurementArea.observe(change_f, "value")
        w = widgets.interactive(
            plot_density_velocity, Id=MeasurementArea, frange=frameRange
        )
        out = widgets.interactive_output(
            plot_density_velocity,
            {"Id": MeasurementArea, "frange": frameRange},
        )
        ui = widgets.HBox([MeasurementArea, frameRange])
        display(ui, out)

    elif choice == "IFD":
        pids = get_ids()
        fm, fM = get_frame_limits(list(ids_IFD)[0])
        frameRange = widgets.IntRangeSlider(
            value=[fm, fM],
            min=fm,
            max=fM,
            step=1,
            description="Frame range",
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        MeasurementArea = widgets.Dropdown(
            options=ids_IFD,
            layout=l,
            style=style,
            value=list(ids_IFD)[0],
            description="Measurement area:",
            disabled=False,
        )
        PedId = widgets.Dropdown(
            options=pids[str(list(ids_IFD)[0])],
            layout=l,
            description="Agent:",
            disabled=False,
        )

        def change_i(*args):
            if str(PedId.value) == "all":
                frameRange.disabled = False
            else:
                frameRange.disabled = True

        def change_x(*args):
            PedId.options = pids[str(MeasurementArea.value)]
            fm, fM = get_frame_limits(MeasurementArea.value)
            frameRange.min = fm
            frameRange.max = fM
            frameRange.value = [fm, fM]

        MeasurementArea.observe(change_x, "value")
        PedId.observe(change_i, "value")
        out = widgets.interactive_output(
            plot_IFD_density_velocity,
            {"Id": MeasurementArea, "pid": PedId, "frange": frameRange},
        )
        ui = widgets.HBox([MeasurementArea, PedId, frameRange])
        display(ui, out)

    elif choice == "Profiles":
        files = glob.glob(os.path.join(field_dir, "density", "Prf_*.dat"))
        ids = set([])
        for f in files:
            ids.add(int(f.split("id_")[-1].split("_")[0]))
        sorted(ids)
        MeasurementArea = widgets.Dropdown(
            options=ids,
            layout=l,
            style=style,
            description="Measurement area:",
            disabled=False,
        )
        out = widgets.interactive_output(
            plot_profiles, {"Id": MeasurementArea}
        )
        ui = widgets.HBox([MeasurementArea, out])
        display(ui)

    elif choice == "Voronoi polygons":
        pids = get_ids()
        l = widgets.Layout(flex="0 1 auto", width="160px")
        fm, fM = get_frame_limits(list(ids_IFD)[0])
        frame = widgets.IntSlider(
            value=fm,
            min=fm,
            max=fM,
            step=1,
            description="Frame: ",
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        MeasurementArea = widgets.Dropdown(
            options=ids_IFD,
            layout=l,
            style=style,
            value=list(ids_IFD)[0],
            description="Measurement area:",
            disabled=False,
        )

        def change_x(*args):
            fm, fM = get_frame_limits(MeasurementArea.value)
            frame.min = fm
            frame.max = fM
            frame.value = fm

        MeasurementArea.observe(change_x, "value")
        out = widgets.interactive_output(
            IFD_plot_polygon_rho, {"Id": MeasurementArea, "frame": frame}
        )
        ui = widgets.HBox([MeasurementArea, frame])
        display(ui, out)

    else:
        print(">> all")
        # todo?

In [ ]:
if method_D.lower() == "true":
    choice = widgets.Dropdown(
        options=["FD", "IFD", "Profiles", "Voronoi polygons"],
        value="FD",
        description="Plot: ",
        disabled=False,
    )
    widgets.interact(
        plot_method_D, choice=choice, description="Choice", disabled=False
    )

else:
    print("Method D not active. Nothing to plot..")